In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Logistic').getOrCreate()

24/11/01 02:17:49 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.0.0.176 instead (on interface wlp4s0)
24/11/01 02:17:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/01 02:17:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/01 02:17:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
from pyspark.ml.classification import LogisticRegression

In [7]:
df = spark.read.format('libsvm').load('datasets/sample_libsvm_data.txt')

24/11/01 02:17:55 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [9]:
my_log_reg_model = LogisticRegression()

In [10]:
trianed_model = my_log_reg_model.fit(df)

In [11]:
log_summary = trianed_model.summary

In [12]:
log_summary.predictions

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [13]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [14]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514862...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198867...|[6.76550380001560...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678715831...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012719...|[4.62137287298722...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874697...|[1.81823629113437...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504187...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212813...|[6.97903542824686...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503566...|[3.00582577441380...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606570...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [15]:
train_data, test_data = df.randomSplit([0.7, 0.3])

In [17]:
new_model = LogisticRegression().fit(train_data)

In [18]:
evaluation = new_model.evaluate(test_data)

In [23]:
evaluation.predictions.show()


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[17.8786545837697...|[0.99999998280512...|       0.0|
|  0.0|(692,[123,124,125...|[19.3732316212302...|[0.99999999614244...|       0.0|
|  0.0|(692,[124,125,126...|[25.2281102870766...|[0.99999999998894...|       0.0|
|  0.0|(692,[126,127,128...|[30.1113878470059...|[0.99999999999991...|       0.0|
|  0.0|(692,[126,127,128...|[14.1086160444417...|[0.99999925405710...|       0.0|
|  0.0|(692,[126,127,128...|[13.9735712917421...|[0.99999914620280...|       0.0|
|  0.0|(692,[126,127,128...|[21.7691313572345...|[0.99999999964861...|       0.0|
|  0.0|(692,[126,127,128...|[27.3078100487225...|[0.99999999999861...|       0.0|
|  0.0|(692,[128,129,130...|[23.1848863267119...|[0.99999999991470...|       0.0|
|  0.0|(692,[152

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [25]:
my_eval = BinaryClassificationEvaluator()

In [27]:
my_finala_roc = my_eval.evaluate(evaluation.predictions)

In [28]:
my_finala_roc

0.9500000000000001